### Importing the necessary libraries:

In [2]:
from pyteomics import mgf, parser
from Bio.SeqUtils import ProtParam

###  Read in the Spectral File

In [4]:
spectral_file = "data/spectral_file.mgf"

with mgf.read(spectral_file) as spectra:
    for spectrum in spectra:
        # extract relevant information from each spectrum
        scan_num = spectrum['params']['title']
        precursor_mz = spectrum['params']['pepmass'][0]
        precursor_charge = spectrum['params']['charge'][0]
        peaks = spectrum['m/z array']
        intensities = spectrum['intensity array']
        # process peaks and intensities...

###  Read in the peptide sequence file 

In [5]:
peptide_file = 'data/psmlist.txt'

# Ion types to consider
ion_types = ['b', 'y']

# Fragment charge states to consider
charge_states = [1, 2]

# Fragment mass tolerance (in Da)
fragment_tol = 0.5

peptide_dict = {}

In [14]:
with open(peptide_file, 'r') as f:
    for line in f:
        SpecFile, ScanNum, PrecursorMZ, Charge, Peptide = line.strip().split('\t')
        peptide_dict[Peptide] = (SpecFile, ScanNum, PrecursorMZ, Charge, Peptide)

ValueError: not enough values to unpack (expected 5, got 1)

In [ ]:
for peptide, (spec_file, scan_num, precursor_mz, charge) in peptide_dict.items():
   # Print some information about the current peptide
    print(f"Processing peptide: {peptide}, SpecFile: {spec_file}, ScanNum: {scan_num}")

    # Convert precursor_mz and charge to floats
    precursor_mz = float(precursor_mz)
    charge = int(charge)

    # Calculate the precursor mass from m/z and charge
    precursor_mass = precursor_mz * charge - charge * mass.proton

    # Generate theoretical peak list for each ion type and charge state
    for ion_type in ion_types:
        for c in range(1, charge + 1):
            ion_series = ion_type + str(c) + '+'
            ion_masses = mass.fast_mass(
                ion_series,
                seq=peptide,
                ion_type=ion_type,
                charge=c,
                aa_mass=mass.std_aa_mass,
                terminus_mass=mass.std_terminus_mass,
                ion_masses={'y': mass.fast_mass('y', seq=peptide, ion_type='y', charge=c, aa_mass=mass.std_aa_mass, terminus_mass=mass.std_terminus_mass)}
            )

            # Filter theoretical peaks based on fragment mass tolerance
            for i, ion_mass in enumerate(ion_masses):
                for neutral_loss in [0.0, mass.water, mass.ammonia]:
                    ion_mass_nl = ion_mass - neutral_loss
                    if abs(ion_mass_nl - precursor_mass) <= fragment_tol:
                        # Print annotated peak for current ion type, charge state, and neutral loss
                        print(f"{ion_series}\t{ion_mass_nl:.4f}\t{ion_mass_nl - precursor_mass:.4f}")


### Generate the theoretical peak list for each peptide:

In [3]:
def get_fragment_masses(peptide, ion_type):
    """
    Calculate the expected masses of fragment ions for a given peptide sequence and ion type
    """
    prot_param = ProtParam.ProteinAnalysis(str(peptide))
    aa_masses = prot_param.monoisotopic_counts

    if ion_type == 'y':
        ion_masses = [sum(aa_masses[i:]) + 19.0178 for i in range(len(aa_masses))]  # add mass of H2O
    elif ion_type == 'b':
        ion_masses = [sum(aa_masses[:i]) + 1.0078 for i in range(len(aa_masses))]  # add mass of H

    return ion_masses


def get_peak_list(peptide, ion_type, fragment_tol):
    """
    Generate a theoretical peak list for a given peptide sequence, ion type, and fragment mass tolerance
    """
    fragment_masses = get_fragment_masses(peptide, ion_type)
    peak_list = []

    for i, ion_mass in enumerate(fragment_masses):
        if i == 0:
            continue
        diff = ion_mass - fragment_masses[i-1]
        if abs(diff - 1.0078) <= fragment_tol:
            peak_list.append({'ion_type': ion_type, 'ion_num': i, 'mass': ion_mass, 'intensity': 1.0})

    return peak_list


theoretical_peak_lists = []
for peptide in peptides:
    peak_list = get_peak_list(peptide['peptide_seq'], ion_type, fragment_tol)
    theoretical_peak_lists.append({'scan_num': peptide['scan_num'], 'peak_list': peak_list})


### Annotate the peaks in the spectral file:

In [4]:
def annotate_peak_list(spectrum_file: str, peptide_file: str, ion_type: str, fragment_tol: float) -> None:
    
    def get_fragment_ions(peptide: str, ion_type: str) -> Tuple[list, list]:
        """
        Generate a list of theoretical fragment ions for a given peptide sequence and ion type.
        Returns two lists: fragment ion names and their corresponding masses.
        """
        ion_masses = []
        ion_names = []
        aa_masses = mass.std_aa_mass
        prefix = ion_type[0]
        suffix = ion_type[1]
        charge = ion_type[2]
        for i in range(1, len(peptide)):
            ion_mass = sum(aa_masses[aa] for aa in peptide[:i])
            ion_names.append(prefix + str(i) + suffix + charge)
            ion_masses.append(ion_mass / charge)
            if suffix == 'Y':
                ion_masses[-1] += mass.H2O + mass.proton
                ion_names.append(ion_names[-1] + '-H2O')
                ion_masses.append(ion_masses[-1] + mass.H2O)
                ion_names.append(ion_names[-2] + '-NH3')
                ion_masses.append(ion_masses[-2] + mass.NH3)
            else:
                ion_names.append(ion_names[-1] + '+H2O')
                ion_masses.append(ion_masses[-1] + mass.H2O + mass.proton)
                ion_names.append(ion_names[-2] + '+NH3')
                ion_masses.append(ion_masses[-2] + mass.NH3 + mass.proton)
        return ion_names, ion_masses
    
    with open(peptide_file) as f:
        for line in f:
            spec_file, scan_num, precursor_mz, charge, peptide = line.strip().split('\t')
            precursor_mz = float(precursor_mz)
            charge = int(charge)
            spectrum = mgf.read(spectrum_file, spectrum_id=scan_num)
            spectrum = next(spectrum)
            spectrum = {k:v for k,v in spectrum.items() if k.startswith('m/z array') or k.startswith('intensity array')}
            mzs = spectrum['m/z array']
            intensities = spectrum['intensity array']
            ion_names, ion_masses = get_fragment_ions(peptide, ion_type)
            annotated_peaks = []
            for name, mass in zip(ion_names, ion_masses):
                for i, mz in enumerate(mzs):
                    if abs(mz - mass) <= fragment_tol:
                        annotated_peaks.append((name, mz, intensities[i]))
            with open(f"{spec_file}_{scan_num}.txt", 'w') as out_file:
                out_file.write("BEGIN\n")
                out_file.write(f"PEPTIDE={peptide}\n")
                out_file.write(f"TITLE={spec_file}_{scan_num}\n")
                for peak in annotated_peaks:
                    out_file.write(f"{peak[0]}\t{peak[1]}\t{peak[2]}\n")
                out_file.write("END\n")


SyntaxError: unexpected EOF while parsing (948253125.py, line 8)

## All at once

In [ ]:
import re
# Parse spectral file and peptide sequence file
def parse_input_files(spectral_file, peptide_file):
    spectra = {}
    with open(spectral_file, 'r') as f:
        spectrum_lines = f.read().splitlines()
    i = 0
    while i < len(spectrum_lines):
        if spectrum_lines[i].startswith('BEGIN IONS'):
            scan_num = None
            precursor_mz = None
            charge = None
            peaks = []
            i += 1
            while i < len(spectrum_lines) and not spectrum_lines[i].startswith('END IONS'):
                line = spectrum_lines[i]
                if line.startswith('TITLE='):
                    scan_num = int(re.findall(r'Scan (\d+)', line)[0])
                elif line.startswith('PEPMASS='):
                    precursor_mz = float(line.split('=')[1])
                elif line.startswith('CHARGE='):
                    charge = int(line.split('=')[1].replace('+', ''))
                elif line.count('\t') == 1:
                    mz, intensity = line.split('\t')
                    peaks.append((float(mz), float(intensity)))
                i += 1
            spectra[scan_num] = {'precursor_mz': precursor_mz, 'charge': charge, 'peaks': peaks}
    
    peptides = []
    with open(peptide_file, 'r') as f:
        peptide_lines = f.read().splitlines()
    for line in peptide_lines:
        spec_file, scan_num, precursor_mz, charge, sequence = line.split('\t')
        scan_num = int(scan_num)
        precursor_mz = float(precursor_mz)
        charge = int(charge)
        peptides.append({'spec_file': spec_file, 'scan_num': scan_num, 'precursor_mz': precursor_mz, 'charge': charge, 'sequence': sequence})
    
    return spectra, peptides

# Generate theoretical peak list for a given peptide
def generate_theoretical_peak_list(peptide, ion_type, fragment_mass_tolerance):
    aa_masses = {
        'G': 57.02146,
        'A': 71.03711,
        'S': 87.03203,
        'P': 97.05276,
        'V': 99.06841,
        'T': 101.04768,
        'C': 103.00919,
        'L': 113.08406,
        'I': 113.08406,
        'X': 113.08406, # Unknown amino acid, treated as Leu
        'N': 114.04293,
        'D': 115.02694,
        'Q': 128.05858,
        'K': 128.09496,
        'E': 129.04259,
        'M': 131.04049,
        'H': 137.05891,
        'F': 147.06841,
        'R': 156.10111,
        'Y': 163.06333,
